# 02 Data Preparation
Export only 80s and onward US movies for further steps

In [24]:
# import the required python packages
import os

import warnings

import pandas as pd
from dotenv import load_dotenv, find_dotenv

# ignore warnings for better output in the notebook
warnings.filterwarnings('ignore')

# load the environment variables from the .env file
_ = load_dotenv(find_dotenv())

#### Import the dataset from the '/data' folder

In [25]:
load_input_file = os.getenv('INPUT_FILE')
df = pd.read_csv(load_input_file, sep=',')

In [26]:
# fix column names
df.columns = df.columns.str.replace(' ', '_').str.replace('/', '_').str.lower()
for col in df.columns:
    print(col)

## Filter the dataset
1980s and onward US movies

In [27]:
df = df[df['release_year'] >= 1980]
df = df[df['origin_ethnicity'] == 'American']

In [28]:
# get only us
print("Number of US movies from 80s and onward: ", df.shape[0])

In [29]:
columns_without_release_year = df.columns[df.columns != 'release_year']

## Data Cleaning

### Fix missing values

In [30]:
# fix missing values like NaN for columns
missing_values_list = df.columns[df.isna().any()].tolist()

In [31]:
for column in missing_values_list:
    # fix missing values like NaN for columns
    df[column] = df[column].fillna('Unknown')
    missing_values = df[column].isna().sum()
    print("After fixing missing values there are {} missing values in the {} column".format(missing_values, column))

In [32]:
df.columns[df.isna().any()].tolist()

All missing values are fixed

### Fix empty strings

In [33]:
for column in columns_without_release_year:
    df[column] = df[column].apply(lambda x: "Unknown" if isinstance(x, str) and len(x.strip()) == 0 else x)
    empty_strings = df[df[column].str.strip() == ''].shape[0]
    print("After fixing empty strings there are {} movies with an empty {} column".format(empty_strings, column))

### Fix multiple values

In [34]:
# fix multiple values
list_with_separators = [" and ", "&", "/", "\r\n", ";"]
replace_values = ", "
columns = ["director", "cast", "genre"]
for column in columns:
    for separator in list_with_separators:
        df[column] = df[column].str.replace(separator, replace_values, regex=False)
        multiple_values = df[column].str.contains(replace_values).sum()
        num = df[column].str.contains(separator).sum()
        print("After fixing {} there are {} left.".format(separator, num))

### Fix special characters

In [35]:
df['plot'] = df['plot'].replace(to_replace=r'\n', value='\\n', regex=True)

### Add Plot Length

In [36]:
df['plot_length'] = df['plot'].apply(lambda x: len(x))

## Display the first 20 rows of the cleaned dataset

In [37]:
df.head(100)

In [38]:
# display 1000 random rows
df.sample(1000)

# EXPORT DATA
Write dataframes to csv files

In [39]:
# export all for live demo
write_output_file = os.getenv('OUTPUT_FILE')
df.to_json(write_output_file, orient='records', lines=False)

In [40]:
# export the 1000 random rows to a json file
df_all = pd.read_json(write_output_file)
df_thousand = df_all.sample(1000)
df_thousand.to_json(
    write_output_file.replace('.json', '_1000_random.json'),
    orient='records',
    lines=False
)

In [41]:
# load the df with 1000 examples to extract 200 random examples
df_two_hundred = df_thousand.sample(200)
df_two_hundred.to_json(
    write_output_file.replace('.json', '_200_random.json'),
    orient='records',
    lines=False
)

In [42]:
# load the df with 200 examples to extract 100 random examples
df_one_hundred = df_two_hundred.sample(100)
df_one_hundred.to_json(
    write_output_file.replace('.json', '_100_random.json'),
    orient='records',
    lines=False
)

In [43]:
# load the df with 100 examples to extract 10 random examples
df_ten = df_one_hundred.sample(10)
df_ten.to_json(
    write_output_file.replace('.json', '_10_random.json'),
    orient='records',
    lines=False
)

In [44]:
# load the df with 10 examples to extract 2 random examples
df_two = df_ten.sample(2)
df_two.to_json(
    write_output_file.replace('.json', '_2_random.json'),
    orient='records',
    lines=False
)

In [45]:
# load the df with 2 examples to extract 1 random example
df_one = df_two.sample(1)
df_one.to_json(
    write_output_file.replace('.json', '_1_random.json'),
    orient='records',
    lines=False
)